In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor, Pool
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn import metrics as metrics
from sklearn.preprocessing import MinMaxScaler
from category_encoders import TargetEncoder  # Другие варианты: CatBoostEncoder, WOEEncoder
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../../datasets/insurance_dataset/insurance.csv")
target = "charges"

num_features = list(df.select_dtypes(exclude='object'))
num_features.remove(target)
cat_features = list(df.select_dtypes(include='object'))
catboost_features = num_features+cat_features

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    df[catboost_features], 
    df[target], 
    test_size=0.33, random_state=2025)
X_train, X_test, y_train, y_test = X_train.reset_index(drop=True), X_test.reset_index(drop=True), y_train.reset_index(drop=True), y_test.reset_index(drop=True)

# 3. Кодирование категориальных признаков
encoder = TargetEncoder(cols=cat_features)
X_train_encoded = encoder.fit_transform(X_train, y_train)
X_test_encoded = encoder.transform(X_test)

scaler = MinMaxScaler()
scaled_X_train_encoded = scaler.fit_transform(X_train_encoded)
scaled_X_train_encoded = pd.DataFrame(scaled_X_train_encoded , columns=catboost_features)
df_train = pd.concat([scaled_X_train_encoded, y_train], axis=1)

scaled_X_test_encoded = scaler.transform(X_test_encoded[catboost_features])
scaled_X_test_encoded = pd.DataFrame(scaled_X_test_encoded , columns=catboost_features)
df_test = pd.concat([scaled_X_test_encoded, y_test], axis=1)


In [4]:
cat_model = CatBoostRegressor(
    num_trees=1,
    depth=3,
    cat_features=cat_features,  # Указываем индексы категориальных признаков
    verbose=0
)
cat_model.fit(X_train, y_train)  # Оригинальные данные (CatBoost сам обрабатывает категории)

In [5]:
# 5. Получение индексов листьев (на закодированных данных)
train_leaf_indices = cat_model.calc_leaf_indexes(Pool(X_train, y_train, cat_features=cat_features)).flatten()
test_leaf_indices = cat_model.calc_leaf_indexes(Pool(X_test, cat_features=cat_features)).flatten()

In [6]:
def get_leaf_stats(leaf_indices, X, y):
    stats_list = []
    for leaf in np.unique(leaf_indices):
        mask = leaf_indices == leaf
        stat = {
            'leaf': leaf,
            'n_samples': np.sum(mask),
            'target_mean': np.mean(y[mask]),
            'target_std': np.std(y[mask])
        }
        
        for col in X.columns:
            stat[f'{col}_mean'] = np.mean(X.loc[mask, col])
            stat[f'{col}_std'] = np.std(X.loc[mask, col])
        
        stats_list.append(stat)
    
    return pd.DataFrame(stats_list)

leaf_stats_df = get_leaf_stats(train_leaf_indices, df_train[catboost_features], df_train[target])

In [7]:
# 7. Кластеризация листьев
cluster_ids = ['leaf', 'n_samples']
cluster_features = [col for col in leaf_stats_df.columns 
                   if not col in cluster_ids]

leaf_scaler = MinMaxScaler()
scaled_leaf_stats_df = leaf_scaler.fit_transform(leaf_stats_df[cluster_features])
scaled_leaf_stats_df = pd.DataFrame(scaled_leaf_stats_df, columns=cluster_features)
scaled_leaf_stats_df = pd.concat([leaf_stats_df[cluster_ids], scaled_leaf_stats_df], axis=1)

scaled_leaf_stats_df

kmeans = KMeans(n_clusters=3, random_state=2025)
scaled_leaf_stats_df['cluster'] = kmeans.fit_predict(scaled_leaf_stats_df[cluster_features])

In [8]:
# 8. Сопоставление кластеров
df_train['leaf'] = train_leaf_indices
df_train = df_train.merge(scaled_leaf_stats_df[['leaf', 'cluster']], on='leaf')

In [9]:
df_train 

,age,bmi,children,sex,smoker,region,charges,leaf,cluster
0,0.130435,0.220766,0.0,0.0,0.0,0.000000,2842.76075,0,0
1,0.521739,0.277372,0.4,1.0,0.0,0.209375,7729.64575,0,0
2,0.173913,0.305229,0.0,1.0,1.0,1.000000,17043.34140,1,0
3,0.108696,0.288843,0.0,1.0,0.0,1.000000,1815.87590,0,0
4,0.717391,0.517950,0.2,0.0,0.0,0.301857,9872.70100,6,0
...,...,...,...,...,...,...,...,...,...
891,0.521739,0.291524,0.0,0.0,1.0,0.000000,21348.70600,1,0
892,0.173913,0.424549,0.0,1.0,0.0,0.000000,2699.56835,6,0
893,0.847826,0.718904,0.0,1.0,0.0,0.209375,11566.30055,6,0
894,0.804348,0.498138,0.4,0.0,0.0,0.000000,12269.68865,6,0


In [10]:
# 9. Обучение линейных моделей
linear_models = {}
for cluster in sorted(scaled_leaf_stats_df['cluster'].unique()):
    cluster_data = df_train[df_train['cluster'] == cluster]
    X_cluster = cluster_data.drop([target, 'leaf', 'cluster'], axis=1)
    print(X_cluster.shape)
    y_cluster = cluster_data[target]
    
    lr = LinearRegression()
    lr.fit(X_cluster, y_cluster)
    linear_models[cluster] = lr

(794, 6)
(99, 6)
(3, 6)


In [17]:
df_train.cluster.value_counts()

cluster
0    794
1     99
2      3
Name: count, dtype: int64

In [18]:
df_test.cluster.value_counts()

cluster
0    388
1     53
2      1
Name: count, dtype: int64

In [12]:
# 10. Предсказание на тесте
df_test['leaf'] = test_leaf_indices
df_test = df_test.merge(scaled_leaf_stats_df[['leaf', 'cluster']], on='leaf', how='left')
df_test['cluster'] = df_test['cluster'].fillna(-1)  # Новые листья

test_predictions = np.zeros(len(X_test))
for cluster, model in linear_models.items():
    mask = df_test['cluster'] == cluster
    if mask.sum() > 0:
        test_predictions[mask] = model.predict(df_test.loc[mask, X_test_encoded.columns])

In [13]:
print(f"Test R2: {metrics.r2_score(y_test, test_predictions)}")
print(f"Test MSE: {metrics.mean_squared_error(y_test, test_predictions)}")
print(f"Test RMSE: {np.sqrt(metrics.mean_squared_error(y_test, test_predictions))}")
print(f"Test MAE: {metrics.mean_absolute_error(y_test, test_predictions)}")
print(f"Test MAPE: {metrics.mean_absolute_percentage_error(y_test, test_predictions)}")

Test R2: 0.8517629026483037
Test MSE: 22187732.201694086
Test RMSE: 4710.385568262335
Test MAE: 2585.090583981729
Test MAPE: 0.23793470331109223


In [14]:
train_predictions = np.zeros(len(X_train))
for cluster, model in linear_models.items():
    mask = df_train['cluster'] == cluster
    if mask.sum() > 0:
        train_predictions[mask] = model.predict(df_train.loc[mask, X_train_encoded.columns])

In [15]:
print(f"Train R2: {metrics.r2_score(y_train, train_predictions)}")
print(f"Train MSE: {metrics.mean_squared_error(y_train, train_predictions)}")
print(f"Train RMSE: {np.sqrt(metrics.mean_squared_error(y_train, train_predictions))}")
print(f"Train MAE: {metrics.mean_absolute_error(y_train, train_predictions)}")
print(f"Train MAPE: {metrics.mean_absolute_percentage_error(y_train, train_predictions)}")

Train R2: 0.8611226527781969
Train MSE: 20115960.8069833
Train RMSE: 4485.0820290138845
Train MAE: 2357.2248588203397
Train MAPE: 0.23732949232528186
